<a href="https://colab.research.google.com/github/sanjulc17/restaurant_score_analysis/blob/main/Restaurant_Score_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the required Libs

In [ ]:
import pandas as pd
import scipy
import datetime
import numpy as np

## Getting the data

In [ ]:
!ls

sample_data


In [ ]:
df = pd.read_csv("restaurant-scores-lives-standard.csv")

FileNotFoundError: ignored

## Getting a look at data, analysing the datatypes and some useful stats of the data

In [ ]:
df.tail()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
52310,39937,Chantal Guillon,437 A Hayes St,San Francisco,CA,94102,37.779847,-122.418985,"{'needs_recoding': False, 'latitude': '37.7798...",NaN,39937_20160926,2016-09-26T00:00:00,NaN,Reinspection/Followup,39937_20160926_103130,Inadequate sewage or wastewater disposal,Moderate Risk
52311,88303,Kushi Tsuru,1737 Post St 380,San Francisco,CA,94115,NaN,NaN,NaN,1.415594e+10,88303_20170313,2017-03-13T00:00:00,85.0,Routine - Unscheduled,88303_20170313_103161,Low risk vermin infestation,Low Risk
52312,86647,Da Cafe,407 Clement St,San Francisco,CA,94118,NaN,NaN,NaN,NaN,86647_20160504,2016-05-04T00:00:00,NaN,New Ownership,86647_20160504_103116,Inadequate food safety knowledge or lack of ce...,Moderate Risk
52313,97277,OYE MANAGUA,3385 Mission St,San Francisco,CA,94110,NaN,NaN,NaN,NaN,97277_20180816,2018-08-16T00:00:00,85.0,Routine - Unscheduled,97277_20180816_103154,Unclean or degraded floors walls or ceilings,Low Risk
52314,86133,Jamba Juice,152 Kearny St,San Francisco,CA,94108,NaN,NaN,NaN,NaN,86133_20160727,2016-07-27T00:00:00,92.0,Routine - Unscheduled,86133_20160727_103120,Moderate risk food holding temperature,Moderate Risk


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52315 entries, 0 to 52314
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   business_id            52315 non-null  int64  
 1   business_name          52315 non-null  object 
 2   business_address       52315 non-null  object 
 3   business_city          52315 non-null  object 
 4   business_state         52315 non-null  object 
 5   business_postal_code   51069 non-null  object 
 6   business_latitude      29530 non-null  float64
 7   business_longitude     29530 non-null  float64
 8   business_location      29530 non-null  object 
 9   business_phone_number  16153 non-null  float64
 10  inspection_id          52315 non-null  object 
 11  inspection_date        52315 non-null  object 
 12  inspection_score       38590 non-null  float64
 13  inspection_type        52315 non-null  object 
 14  violation_id           39646 non-null  object 
 15  vi

In [ ]:
df.describe()

,business_id,business_latitude,business_longitude,business_phone_number,inspection_score
count,52315.000000,29530.000000,29530.000000,1.615300e+04,38590.000000
mean,50661.368078,37.743790,-122.336773,1.415540e+10,85.953563
std,35423.666201,1.030884,3.340570,1.225992e+06,8.777229
min,19.000000,0.000000,-122.510896,1.415020e+10,45.000000
25%,5855.000000,37.755282,-122.437717,1.415537e+10,81.000000
50%,65925.000000,37.780348,-122.419558,1.415561e+10,87.000000
75%,82217.000000,37.789544,-122.408345,1.415578e+10,92.000000
max,98377.000000,37.824494,0.000000,1.415988e+10,100.000000


## 2.1: Number of violations aggregated by year
### Which restaurant(s) have the highest number of violations for 2018?


In [ ]:
 # Creating a year column
 df["inspection_year"] = df.apply(lambda x: datetime.datetime.strptime(x["inspection_date"], '%Y-%m-%dT%H:%M:%S').year, axis=1)

In [ ]:
# Creating a new DF with only 2018's violations
df2 = df.loc[(df['inspection_year'] == 2018) & (pd.isna(df['violation_id']) == False)][['inspection_year', 'business_name']].groupby(['business_name']).count()#.nlargest(2, "inspection_year")

In [ ]:
df2.loc[df2['inspection_year'] == df2.max().values[0]]

,inspection_year
business_name,
McDonald's,20
Peet's Coffee & Tea,20


###Which restaurant(s) have the lowest number of violations for 2017?

In [ ]:
df2 = df.loc[(df['inspection_year'] == 2017) & (pd.isna(df['violation_id']) == False)]\
  [['inspection_year', 'business_name']]\
  .groupby(['business_name'])\
  .count()

In [ ]:
# Since violations are asked it means there must be 1 minimum violation
df2.loc[df2['inspection_year'] == 1] 

,inspection_year
business_name,
17th & Noe Market,1
"20 Spot Mission, LLC",1
"24 Hour Fitness Club, #273",1
24TH STREET CHEESE COMPANY,1
3RD BAPTIST FELLOWSHIP HALL,1
...,...
Yick Wo Elementary School,1
Your Market,1
Zaida T. Rodriguez (ZTR) Annex Child Development Center,1


##  2.2: Inspection score changes over time
### What was the change in average inspection score by postal code between 2016 and 2017?


In [ ]:
def cal_avg_score_by_year(df, year):
  return df.loc[(df['inspection_year'] == year) & (pd.isna(df['inspection_score']) == False)]\
    [['inspection_score', 'business_postal_code']]\
    .groupby(['business_postal_code'])\
    .mean()

In [ ]:
df_avg_2016 = cal_avg_score_by_year(df, 2016)

In [ ]:
df_avg_2017 = cal_avg_score_by_year(df, 2017)

In [ ]:
df_avg_changes=df_avg_2017 - df_avg_2016
df_avg_changes

,inspection_score
business_postal_code,
00000,NaN
92672,NaN
94013,NaN
94014,NaN
94080,NaN
94101,10.000000
94102,1.367008
94103,0.387719
941033148,-12.000000


### Which postal code(s) had the greatest positive change?

In [ ]:
df_avg_changes.sort_values('inspection_score', ascending=False).iloc[0]

inspection_score    10.0
Name: 94101, dtype: float64

change is positive: 10
postal code is: 94101

### Which postal code(s) had the greatest negative change?

In [ ]:
df_avg_changes.sort_values('inspection_score', ascending=True).iloc[0]

inspection_score   -12.0
Name: 941033148, dtype: float64

change is negative: -12 postal code is: 941033148